In [3]:
import csv
from itertools import islice

with open('personal_info.csv') as f:
    sample = f.read(2000)
    dialect = csv.Sniffer().sniff(sample)
print(vars(dialect))

print()

with open('personal_info.csv') as f:
    reader = csv.reader(f, dialect)
    for row in islice(reader, 5):
        print(row)

{'__module__': 'csv', '_name': 'sniffed', 'lineterminator': '\r\n', 'quoting': 0, '__doc__': None, 'doublequote': False, 'delimiter': ',', 'quotechar': '"', 'skipinitialspace': False}
['ssn', 'first_name', 'last_name', 'gender', 'language']
['100-53-9824', 'Sebastiano', 'Tester', 'Male', 'Icelandic']
['101-71-4702', 'Cayla', 'MacDonagh', 'Female', 'Lao']
['101-84-0356', 'Nomi', 'Lipprose', 'Female', 'Yiddish']
['104-22-0928', 'Justinian', 'Kunzelmann', 'Male', 'Dhivehi']


In [26]:
f_names = 'cars.csv', 'personal_info.csv'
import csv
from itertools import islice

def get_dialect(f_name):
    with open(f_name) as f:
        return csv.Sniffer().sniff(f.read(1000))

for file in f_names:
    with open(file) as f:
        sample = f.read(2000)
        dialect = csv.Sniffer().sniff(sample)
    print('\n-----------------')


-----------------

-----------------


In [27]:
import csv
from collections import namedtuple

class File():
    def __init__(self, filename):
        self.name = filename
        
    def __enter__(self):
        self._f = open(self.name, 'r')
        self._reader = csv.reader(self._f, get_dialect(self.name))
        headers = map(lambda x: x.lower(), next(self._reader))
        self._nt = namedtuple('Data', headers)
        return self

    def __iter__(self):
        return self

    def __next__(self):
        if self._f.closed:
            # file has been closed - so we're can't iterate anymore!
            raise StopIteration
        else:
            return self._nt(*next(self._reader))

    def __exit__(self, exc_type, exc_value, exc_traceback):
        self._f.close()
        return False

In [28]:
from itertools import islice

with File('cars.csv') as data:
    for row in islice(data, 10):
        print(row)

print()

with File('personal_info.csv') as data:
    for row in islice(data, 10):
        print(row)

Data(car='Chevrolet Chevelle Malibu', mpg='18.0', cylinders='8', displacement='307.0', horsepower='130.0', weight='3504.', acceleration='12.0', model='70', origin='US')
Data(car='Buick Skylark 320', mpg='15.0', cylinders='8', displacement='350.0', horsepower='165.0', weight='3693.', acceleration='11.5', model='70', origin='US')
Data(car='Plymouth Satellite', mpg='18.0', cylinders='8', displacement='318.0', horsepower='150.0', weight='3436.', acceleration='11.0', model='70', origin='US')
Data(car='AMC Rebel SST', mpg='16.0', cylinders='8', displacement='304.0', horsepower='150.0', weight='3433.', acceleration='12.0', model='70', origin='US')
Data(car='Ford Torino', mpg='17.0', cylinders='8', displacement='302.0', horsepower='140.0', weight='3449.', acceleration='10.5', model='70', origin='US')
Data(car='Ford Galaxie 500', mpg='15.0', cylinders='8', displacement='429.0', horsepower='198.0', weight='4341.', acceleration='10.0', model='70', origin='US')
Data(car='Chevrolet Impala', mpg='14

In [29]:
import csv
from collections import namedtuple

def get_dialect(f_name):
    with open(f_name) as f:
        return csv.Sniffer().sniff(f.read(1000))

class FileParser:
    def __init__(self, f_name):
        self.f_name = f_name
        
    def __enter__(self):
        self._f = open(self.f_name, 'r')
        self._reader = csv.reader(self._f, get_dialect(self.f_name))
        headers = map(lambda x: x.lower(), next(self._reader))
        self._nt = namedtuple('Data', headers)
        return self
        
    def __exit__(self, exc_type, exc_value, exc_tb):
        self._f.close()
        return False
    
    def __iter__(self):
        return self
    
    def __next__(self):
        if self._f.closed:
            # file has been closed - so we're can't iterate anymore!
            raise StopIteration
        else:
            return self._nt(*next(self._reader))



In [30]:
def parsed_data_iter(data_iter, nt):
    for row in data_iter:
        yield nt(*row)  
from contextlib import contextmanager
import csv
from collections import namedtuple
    
@contextmanager
def parsed_data(f_name):
    f = open(f_name, 'r')
    try:
        reader = csv.reader(f, get_dialect(f_name))
        headers = map(lambda x: x.lower(), next(reader))
        nt = namedtuple('Data', headers)
        yield parsed_data_iter(reader, nt)
    finally:
        f.close()    

from itertools import islice

with parsed_data('personal_info.csv') as data:
        for row in islice(data, 5):
            print(row)
            
with parsed_data('cars.csv') as data:
    for row in islice(data, 5):
        print(row)

Data(ssn='100-53-9824', first_name='Sebastiano', last_name='Tester', gender='Male', language='Icelandic')
Data(ssn='101-71-4702', first_name='Cayla', last_name='MacDonagh', gender='Female', language='Lao')
Data(ssn='101-84-0356', first_name='Nomi', last_name='Lipprose', gender='Female', language='Yiddish')
Data(ssn='104-22-0928', first_name='Justinian', last_name='Kunzelmann', gender='Male', language='Dhivehi')
Data(ssn='104-84-7144', first_name='Claudianus', last_name='Brixey', gender='Male', language='Afrikaans')


In [31]:
@contextmanager
def parsed_data(f_name):
    f = open(f_name, 'r')
    try:
        dialect = csv.Sniffer().sniff(f.read(1000))
        f.seek(0)
        reader = csv.reader(f, dialect)
        headers = map(lambda x: x.lower(), next(reader))
        nt = namedtuple('Data', headers)
        yield (nt(*row) for row in reader)
    finally:
        f.close()  
        
f_names = 'cars.csv', 'personal_info.csv'
for f_name in f_names:
    with parsed_data(f_name) as data:
        for row in islice(data, 5):
            print(row)
    print('-------------------')

Data(car='Chevrolet Chevelle Malibu', mpg='18.0', cylinders='8', displacement='307.0', horsepower='130.0', weight='3504.', acceleration='12.0', model='70', origin='US')
Data(car='Buick Skylark 320', mpg='15.0', cylinders='8', displacement='350.0', horsepower='165.0', weight='3693.', acceleration='11.5', model='70', origin='US')
Data(car='Plymouth Satellite', mpg='18.0', cylinders='8', displacement='318.0', horsepower='150.0', weight='3436.', acceleration='11.0', model='70', origin='US')
Data(car='AMC Rebel SST', mpg='16.0', cylinders='8', displacement='304.0', horsepower='150.0', weight='3433.', acceleration='12.0', model='70', origin='US')
Data(car='Ford Torino', mpg='17.0', cylinders='8', displacement='302.0', horsepower='140.0', weight='3449.', acceleration='10.5', model='70', origin='US')
-------------------
Data(ssn='100-53-9824', first_name='Sebastiano', last_name='Tester', gender='Male', language='Icelandic')
Data(ssn='101-71-4702', first_name='Cayla', last_name='MacDonagh', gen